In [ ]:
import openai
openai.api_key = "Mettre clé API OpenAI"

In [ ]:
import pandas as pd


df = pd.read_csv("chemin d'accès")
df


,name,synopsis,genre
0,"Ma Mère, Dieu et Sylvie Vartan","En 1963, Esther met au monde Roland, petit der...",Comédie dramatique
1,Blanche Neige,"""Blanche-Neige"" des studios Disney est une nou...","Aventure, Fantastique, Comédie musicale"
2,"Minecraft, Le Film",Bienvenue dans l’univers de Minecraft où la cr...,"Aventure, Comédie, Famille"
3,Mickey 17,"Héros malgré lui, Mickey Barnes se tue à la tâ...","Action, Comédie, Science Fiction"
4,On Ira,"Marie, 80 ans, en a ras le bol de sa maladie. ...",Comédie dramatique
...,...,...,...
10045,Ça reste entre nous,"Patrick aime son exquise femme, qui lui a fait...",Comédie
10046,Mute,"Avertissement : des scènes, des propos ou des ...","Science Fiction, Thriller"
10047,Le Jour des Corneilles,"Le fils Courge vit au cœur de la forêt, élevé ...",Animation
10048,The Eyes Of My Mother,Marquée par le meurtre de sa mère alors qu'ell...,"Drame, Epouvante-horreur"


In [42]:
import numpy as np
import openai

def embed_batch(texts, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=texts,
        model=model
    )
    return [r.embedding for r in response.data]

# Préparer les textes
df["texte"] = df["synopsis"].fillna("") + " Genres : " + df["genre"].fillna("")
df["texte"] = df["texte"].astype(str)
df["texte"] = df["texte"].apply(lambda x: x[:4000])  # éviter dépassement de tokens

# Initialiser une liste pour stocker les embeddings
all_embeddings = []

# Découper en paquets de 100 textes
batch_size = 100
for i in range(0, len(df), batch_size):
    batch_texts = df["texte"].iloc[i:i+batch_size].tolist()
    try:
        batch_embeddings = embed_batch(batch_texts)
        all_embeddings.extend(batch_embeddings)
    except Exception as e:
        print(f"Erreur au batch {i}-{i+batch_size}: {e}")

# Ajouter les embeddings au DataFrame
if len(all_embeddings) == len(df):
    df["embedding"] = all_embeddings
else:
    print("Erreur : le nombre d'embeddings ne correspond pas au nombre de lignes.")



In [43]:
df.to_csv("films_allocine_parallelfinalavecembed.csv", index=False, encoding="utf-8-sig")

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

def recommander(titre_film, df, top_k=3):
    if titre_film not in df["name"].values:
        print("Film non trouvé.")
        return None

    index = df[df["name"] == titre_film].index[0]
    emb = np.array(df.loc[index, "embedding"]).reshape(1, -1)
    all_embs = np.vstack(df["embedding"].values)
    sims = cosine_similarity(emb, all_embs)[0]

    df["similarité"] = sims
    reco = df[df["name"] != titre_film].sort_values(by="similarité", ascending=False)
    return reco[["name", "genre", "similarité"]].head(top_k)


In [34]:
recommandations = recommander("Interstellar", df)
print(recommandations)


          name                                       genre  similarité
248    O'Dessa  Comédie musicale, Romance, Science Fiction    0.512549
184  Civil War                            Action, Thriller    0.510902
93   The Gorge                   Action, Romance, Thriller    0.510490
